# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
import nltk
nltk.download(['punkt', 'wordnet','omw-1.4'])
import warnings
warnings.simplefilter('ignore')
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix, make_scorer, accuracy_score, precision_score, recall_score, f1_score

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\polym\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\polym\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\polym\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
#load data from database setting up the engine connection sql 
engine = create_engine('sqlite:///C:/Users/polym/DisasterResponse_Database.db')
#load sql table using pandas 
df = pd.read_sql_table('DisasterResponse_Database', engine)
#set the design matrix X using messages as input
X = df['message']
#set the response target y using it as output
y = df[df.columns[4:]]
#outputs the classes of target response y
category_names = y.columns

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    '''
    Returns: clean_tokens
    
    input:
          text: raw text as string   
          
    output:
           clean_tokens: raw text cleaned    
    ''' 

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
#Set the model based on ML pipeline class object
model = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
            ('tfidf',TfidfTransformer()),
            ('clf',MultiOutputClassifier(RandomForestClassifier(class_weight="balanced",random_state=2022)))
            ])

In [5]:
#observe model parameters structure
model.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estim

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3, random_state=2022)

# train classifier based on model
model.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F786BCDAF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        random_state=2022)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
#compute predictions of the target variable y using the test set's design matrix X_test
y_pred = model.predict(X_test)
y_pred.shape

(7865, 36)

In [8]:
#iterating through the columns and calling sklearn's classification_report on each y_pred and y_test
#outputs in dictionary the classes of target response y and the corresponding values
reversefactor = dict(zip(range(36),y_test.columns))
#y_test will be converted into y_true for purposes of illustration metrics
y_true = np.vectorize(reversefactor.get)(y_test)
#y_pred will be converted into y_est for purposes of illustration metrics
y_est = np.vectorize(reversefactor.get)(y_pred)
for i in range(36):
    print('target response class',y_test.columns[i], ':')
    print(classification_report(np.array(y_true)[:,i], y_est[:,i]), '**********************************************************')
    print('Confusion Matrix : \n' + str(pd.crosstab(np.array(y_true)[:,i],y_est[:,i], rownames=['Actual Classes'], colnames=['Predicted Classes'])))

target response class related :
              precision    recall  f1-score   support

       offer       0.32      0.17      0.22        48
     related       0.73      0.26      0.38      1878
     request       0.80      0.97      0.88      5939

    accuracy                           0.79      7865
   macro avg       0.62      0.46      0.49      7865
weighted avg       0.78      0.79      0.75      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  offer  related  request
Actual Classes                            
offer                  8        1       39
related                3      485     1390
request               14      174     5751
target response class request :
              precision    recall  f1-score   support

     related       0.90      0.99      0.94      6524
     request       0.89      0.45      0.60      1341

    accuracy                           0.90      7865
   macro avg       0.89      0.72      0.77

              precision    recall  f1-score   support

     related       0.99      1.00      0.99      7779
     request       0.00      0.00      0.00        86

    accuracy                           0.99      7865
   macro avg       0.49      0.50      0.50      7865
weighted avg       0.98      0.99      0.98      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related
Actual Classes            
related               7779
request                 86
target response class refugees :
              precision    recall  f1-score   support

     related       0.97      1.00      0.98      7594
     request       0.00      0.00      0.00       271

    accuracy                           0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Clas

              precision    recall  f1-score   support

     related       0.99      1.00      1.00      7794
     request       0.00      0.00      0.00        71

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related
Actual Classes            
related               7794
request                 71
target response class earthquake :
              precision    recall  f1-score   support

     related       0.94      1.00      0.97      7116
     request       0.91      0.37      0.52       749

    accuracy                           0.94      7865
   macro avg       0.92      0.68      0.75      7865
weighted avg       0.93      0.94      0.92      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Cl

### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
#specify parameters for grid search
parameters = {
        'clf__estimator__n_estimators': [10,20],
         'clf__estimator__max_depth':[4,5]
}

#set some standard metrics of multi-classification based on make_score() function
scorers = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'macro'),
           'recall': make_scorer(recall_score, average = 'macro'),
           'f1': make_scorer(f1_score, average = 'macro')}

#set model with gridsearch method based on crossvalidation (default=3) and scoring f1-metric
cv = GridSearchCV(estimator=model, param_grid=parameters, cv=3, scoring=scorers, verbose=3,refit='f1')

In [10]:
#train the model with gridsearch method
tuned_model=cv.fit(X_train,y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV 1/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=10; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   8.5s
[CV 2/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=10; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   8.5s
[CV 3/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=10; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=   8.5s
[CV 1/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=20; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=  10.5s
[CV 2/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=20; accuracy: (test=nan) f1: (test=nan) precision: (test=nan) recall: (test=nan) total time=  10.4s
[CV 3/3] END clf__estimator__max_depth=4, clf__estimator__n_estimators=20; accuracy: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
#compute predictions of the target variable y using the test set's design matrix X_test
y_pred = model.predict(X_test)
#iterating through the columns and calling sklearn's classification_report on each y_pred and y_test
#outputs in dictionary the classes of target response y and the corresponding values
reversefactor = dict(zip(range(36),y_test.columns))
#y_test will be converted into y_true for purposes of illustration metrics
y_true = np.vectorize(reversefactor.get)(y_test)
#y_pred will be converted into y_est for purposes of illustration metrics
y_est = np.vectorize(reversefactor.get)(y_pred)
for i in range(36):
    print('target response class',y_test.columns[i], ':')
    print(classification_report(np.array(y_true)[:,i], y_est[:,i]), '**********************************************************')
    print('Confusion Matrix : \n' + str(pd.crosstab(np.array(y_true)[:,i],y_est[:,i], rownames=['Actual Classes'], colnames=['Predicted Classes'])))

target response class related :
              precision    recall  f1-score   support

       offer       0.32      0.17      0.22        48
     related       0.73      0.26      0.38      1878
     request       0.80      0.97      0.88      5939

    accuracy                           0.79      7865
   macro avg       0.62      0.46      0.49      7865
weighted avg       0.78      0.79      0.75      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  offer  related  request
Actual Classes                            
offer                  8        1       39
related                3      485     1390
request               14      174     5751
target response class request :
              precision    recall  f1-score   support

     related       0.90      0.99      0.94      6524
     request       0.89      0.45      0.60      1341

    accuracy                           0.90      7865
   macro avg       0.89      0.72      0.77

Confusion Matrix : 
Predicted Classes  related
Actual Classes            
related               7779
request                 86
target response class refugees :
              precision    recall  f1-score   support

     related       0.97      1.00      0.98      7594
     request       0.00      0.00      0.00       271

    accuracy                           0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Classes                     
related               7593        1
request                271        0
target response class death :
              precision    recall  f1-score   support

     related       0.96      1.00      0.98      7523
     request       0.77      0.03      0.06       342

    accuracy                           0.96      7865
   macro avg       0.86      0.51      0.

              precision    recall  f1-score   support

     related       0.99      1.00      1.00      7794
     request       0.00      0.00      0.00        71

    accuracy                           0.99      7865
   macro avg       0.50      0.50      0.50      7865
weighted avg       0.98      0.99      0.99      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related
Actual Classes            
related               7794
request                 71
target response class earthquake :
              precision    recall  f1-score   support

     related       0.94      1.00      0.97      7116
     request       0.91      0.37      0.52       749

    accuracy                           0.94      7865
   macro avg       0.92      0.68      0.75      7865
weighted avg       0.93      0.94      0.92      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Cl

In [12]:
#best model performance based on grid search method
tuned_model.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F786BCDAF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced',
                                                                        max_depth=4,
                                                                        n_estimators=10,
                                                                        random_state=2022)))])

In [13]:
#best model performance based on grid search method
tuned_model.best_params_

{'clf__estimator__max_depth': 4, 'clf__estimator__n_estimators': 10}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
#instead of random forest classifier, apply multinomial naive Bayes Classifier
model = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),
            ('tfidf',TfidfTransformer()),
            ('clf',MultiOutputClassifier(MultinomialNB()))
            ])

In [15]:
# train classifier based on model
model.fit(X_train,y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F786BCDAF0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=MultinomialNB()))])

In [16]:
#compute predictions of the target variable y using the test set's design matrix X_test
y_pred = model.predict(X_test)
y_pred.shape

(7865, 36)

In [17]:
#iterating through the columns and calling sklearn's classification_report on each y_pred and y_test
#outputs in dictionary the classes of target response y and the corresponding values
reversefactor = dict(zip(range(36),y_test.columns))
#y_test will be converted into y_true for purposes of illustration metrics
y_true = np.vectorize(reversefactor.get)(y_test)
#y_pred will be converted into y_est for purposes of illustration metrics
y_est = np.vectorize(reversefactor.get)(y_pred)
for i in range(36):
    print('target response class',y_test.columns[i], ':')
    print(classification_report(np.array(y_true)[:,i], y_est[:,i]), '**********************************************************')
    print('Confusion Matrix : \n' + str(pd.crosstab(np.array(y_true)[:,i],y_est[:,i], rownames=['Actual Classes'], colnames=['Predicted Classes'])))

target response class related :
              precision    recall  f1-score   support

       offer       0.00      0.00      0.00        48
     related       0.82      0.04      0.08      1878
     request       0.76      1.00      0.86      5939

    accuracy                           0.76      7865
   macro avg       0.53      0.35      0.32      7865
weighted avg       0.77      0.76      0.67      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Classes                     
offer                    4       44
related                 83     1795
request                 14     5925
target response class request :
              precision    recall  f1-score   support

     related       0.85      1.00      0.92      6524
     request       0.88      0.13      0.22      1341

    accuracy                           0.85      7865
   macro avg       0.86      0.56      0.57      7865
weighted avg       0.85 

              precision    recall  f1-score   support

     related       0.97      1.00      0.98      7594
     request       0.00      0.00      0.00       271

    accuracy                           0.97      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.93      0.97      0.95      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related
Actual Classes            
related               7594
request                271
target response class death :
              precision    recall  f1-score   support

     related       0.96      1.00      0.98      7523
     request       0.00      0.00      0.00       342

    accuracy                           0.96      7865
   macro avg       0.48      0.50      0.49      7865
weighted avg       0.91      0.96      0.94      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related
Actual Classes         

              precision    recall  f1-score   support

     related       0.91      1.00      0.95      7116
     request       0.88      0.07      0.13       749

    accuracy                           0.91      7865
   macro avg       0.89      0.53      0.54      7865
weighted avg       0.91      0.91      0.87      7865
 **********************************************************
Confusion Matrix : 
Predicted Classes  related  request
Actual Classes                     
related               7109        7
request                698       51
target response class cold :
              precision    recall  f1-score   support

     related       0.98      1.00      0.99      7695
     request       0.00      0.00      0.00       170

    accuracy                           0.98      7865
   macro avg       0.49      0.50      0.49      7865
weighted avg       0.96      0.98      0.97      7865
 **********************************************************
Confusion Matrix : 
Predicted Class

### 9. Export your model as a pickle file

In [19]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.